In [17]:
using LinearAlgebra
using Base

In [8]:
A = rand(3, 3)

3×3 Matrix{Float64}:
 0.771909  0.693813  0.557481
 0.862245  0.305889  0.251974
 0.694945  0.743723  0.373231

In [9]:
b = rand(3)

3-element Vector{Float64}:
 0.8678934084720591
 0.11196687144423367
 0.6251864440515684

In [30]:
sense = :<=

:<=

For the form of Ax <= b, and Ax >= b (ONLY)

Implementation assumes A has full row rank

In [32]:
function fourier_motzkin_elimination(A::Matrix{Float64}, b::Vector{Float64}, sense::Symbol)
    m, n = size(A)
    #Checking to make sure input matrix A size is correct
    if m < n
        throw(ArgumentError("Input matrix A must have at least as many rows as columns."))
    end
    
    #Used for swapping between <= and >=
    if sense == :<=
        A = -A
        b = -b
    elseif sense != :>=
        throw(ArgumentError("Invalid sense argument."))
    end
        
    for i = 1:n
        for j = i+1:n
            if A[i,i] == 0 && A[j,i] != 0
                # Swap rows to ensure A[i,i] != 0
                A[i,:], A[j,:] = A[j,:], A[i,:]
                b[i], b[j] = b[j], b[i]
            end
            if A[j,i] != 0
                # Eliminate variable x_i from the jth inequality
                alpha = A[i,i] / A[j,i]
                A[j,:] = alpha * A[j,:] - A[i,:]
                b[j] = alpha * b[j] - b[i]
            end
        end
    end
            
    if sense == :<=
        A = -A
        b = -b
    end
            
    # Remove rows where all coefficients are zero
    nonzero_rows = [i for i in 1:size(A,1) if any(A[i,:] .!= 0)]
    A = A[nonzero_rows,:]
    b = b[nonzero_rows]
    
    #A are the coefficients
    #b are the constants
    return A, b
end


fourier_motzkin_elimination (generic function with 3 methods)

In [33]:
coeffs, cons = fourier_motzkin_elimination(A, b, sense)

([0.7719091742977146 0.6938128093094919 0.5574806755557287; 0.0 -0.419971499250648 -0.33190562428204107; 0.0 0.0 1.0414247482789284e15], [0.8678934084720591, -0.7676571116766941, 1.7476259688736035e15])

In [67]:
println("Updated inequalities:")
for i = 1:size(coeffs, 1)
    for j = 1:size(coeffs, 2)
        print(coeffs[i, j])
        print("x")
        print(j)
        if(j != 3)
            print(" + ")
        end
    end
    print(" ")
    println(sense)
    println(cons[i])
end

Updated inequalities:
0.7719091742977146x1 + 0.6938128093094919x2 + 0.5574806755557287x3 <=
0.8678934084720591
0.0x1 + -0.419971499250648x2 + -0.33190562428204107x3 <=
-0.7676571116766941
0.0x1 + 0.0x2 + 1.0414247482789284e15x3 <=
1.7476259688736035e15
